# MeanHamilMinimizer_rigetti_autograd

* Feedback loop between Qubiter and Rigetti QVM
* minimization via autograd

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

/home/jupyter/Notebooks/Quantum/qubiter/jupyter-notebooks
/home/jupyter/Notebooks/Quantum/qubiter


In [2]:
import adv_applications.setup_autograd # do this first!
from adv_applications.MeanHamilMinimizer_rigetti import *

np installed? False
numpy installed? True
autograd.numpy installed? True
loaded OneBitGates, WITH autograd.numpy
pu2 in dir True
pu2 in sys.modules False


In [3]:
from pyquil.quil import Program
from pyquil.api import QVMConnection
from pyquil.gates import *
from pyquil import get_qc

In [4]:
qvm_url = "http://localhost:5000"
compiler_server_address = "tcp://localhost:5555"
forest_url = "https://forest-server.qcs.rigetti.com"
qvm = QVMConnection(endpoint=qvm_url, compiler_endpoint=compiler_server_address)

## First Example (taken from Pennylane docs). 

In [5]:
qc = get_qc('2q-qvm')

In [6]:
num_bits = 2
file_prefix = 'io_folder/mean_hamil_rigetti_test1'
emb = CktEmbedder(num_bits, num_bits)
wr = SEO_writer(file_prefix, emb)
wr.write_Rx(0, rads='#1')
wr.write_Ry(0, rads='#2')
wr.close_files()

In [7]:
wr.print_eng_file()

ROTX	#1	AT	0
ROTY	#2	AT	0



In [8]:
wr.print_pic_file()

|   Rx  
|   Ry  



In [9]:
init_var_num_to_rads = {1: .3, 2: .8}

In [10]:
fun_name_to_fun = None

In [11]:
hamil = QubitOperator('Z0', 1.)
print('hamil=\n', hamil)

hamil=
 1.0 [Z0]


In [12]:
num_samples = 100
rand_seed = 1234
print_hiatus = 1
verbose = False

In [13]:
mini = MeanHamilMinimizer_rigetti(qc, file_prefix, num_bits, hamil,
                init_var_num_to_rads, fun_name_to_fun,
                num_samples=num_samples, rand_seed=rand_seed,
                print_hiatus=print_hiatus, verbose=verbose)

In [14]:
mini.translator.print_aqasm_file()


pg += RX(rads1*(-2), 0)
pg += RY(rads2*(-2), 0)




In [15]:
print(mini.pg)

PRAGMA INITIAL_REWIRING "PARTIAL"
RESET
DECLARE ro BIT[2]
DECLARE rads1 REAL[1]
DECLARE rads2 REAL[1]
RX(rads1*-2) 0
RY(rads2*-2) 0
MEASURE 0 ro[0]
MEASURE 1 ro[1]



In [16]:
mini.find_min(interface='autograd', num_iter=10, descent_rate=.1)

x_val~ (#1, #2)
iter=0, cost=0.080000, pred_cost=-0.024099, x_val=0.300000, 0.800000
iter=1, cost=-0.280000, pred_cost=-0.292577, x_val=0.296703, 0.965738
iter=2, cost=-0.600000, pred_cost=-0.551784, x_val=0.257234, 1.128646
iter=3, cost=-0.680000, pred_cost=-0.764436, x_val=0.194856, 1.271741
iter=4, cost=-0.860000, pred_cost=-0.896213, x_val=0.132062, 1.380452
iter=5, cost=-0.960000, pred_cost=-0.959154, x_val=0.083588, 1.453728
iter=6, cost=-0.940000, pred_cost=-0.984771, x_val=0.051216, 1.499885
iter=7, cost=-1.000000, pred_cost=-0.994446, x_val=0.030971, 1.528101
iter=8, cost=-1.000000, pred_cost=-0.997991, x_val=0.018635, 1.545146
iter=9, cost=-1.000000, pred_cost=-0.999275, x_val=0.011193, 1.555399


## Second, more complicated example

In [17]:
qc = get_qc('4q-qvm')

In [18]:
num_bits = 4
file_prefix = 'io_folder/mean_hamil_rigetti_test2'
emb = CktEmbedder(num_bits, num_bits)
wr = SEO_writer(file_prefix, emb)
wr.write_Ry(2, rads=np.pi/7)
wr.write_Rx(1, rads='#2*.5')
wr.write_Ry(1, rads='-#1*.3')
wr.write_cnot(2, 3)
wr.write_bit_swap(1, 2)
wr.close_files()

In [19]:
wr.print_eng_file()

ROTY	25.714285714285715	AT	2
ROTX	#2*.5	AT	1
ROTY	-#1*.3	AT	1
SIGX	AT	3	IF	2T
SWAP	2	1



In [20]:
wr.print_pic_file()

|   Ry  |   |   
|   |   Rx  |   
|   |   Ry  |   
X---@   |   |   
|   <--->   |   



In [21]:
fun_name_to_fun = None

In [22]:
init_var_num_to_rads = {1: 2.1, 2: 3.4}

In [23]:
hamil = QubitOperator('X1 Y3 X1 Y1 X2', .4) + QubitOperator('Y2 X1', .7)
print('hamil=\n', hamil)

hamil=
 0.7 [X1 Y2] +
0.4 [Y1 X2 Y3]


In [24]:
num_samples = 100
rand_seed = 1234
print_hiatus = 1
verbose = False

In [25]:
mini = MeanHamilMinimizer_rigetti(qc, file_prefix, num_bits, hamil,
                init_var_num_to_rads, fun_name_to_fun,
                num_samples=num_samples, rand_seed=rand_seed,
                print_hiatus=print_hiatus, verbose=verbose)

In [26]:
mini.translator.print_aqasm_file()


pg += RY(-0.8975979010256552, 2)
pg += RX(rads2*.5*(-2), 1)
pg += RY(-rads1*.3*(-2), 1)
pg += CNOT(2, 3)
pg += SWAP(2, 1)




In [27]:
print(mini.pg)

PRAGMA INITIAL_REWIRING "PARTIAL"
RESET
DECLARE ro BIT[4]
DECLARE rads1 REAL[1]
DECLARE rads2 REAL[1]
RY(-2*pi/7) 2
RX(rads2*0.5*-2) 1
RY(-1*rads1*0.3*-2) 1
CNOT 2 3
SWAP 2 1
RY(-pi/2) 1
RX(pi/2) 2
MEASURE 0 ro[0]
MEASURE 1 ro[1]
MEASURE 2 ro[2]
MEASURE 3 ro[3]



In [28]:
mini.find_min(interface='autograd', num_iter=10, descent_rate=.1)

x_val~ (#1, #2)
iter=0, cost=-0.048000, pred_cost=-0.287864, x_val=2.100000, 3.400000
iter=1, cost=-0.104000, pred_cost=-0.288742, x_val=2.105548, 3.392383
iter=2, cost=-0.074000, pred_cost=-0.289583, x_val=2.111049, 3.384978
iter=3, cost=-0.098000, pred_cost=-0.290390, x_val=2.116503, 3.377780
iter=4, cost=-0.078000, pred_cost=-0.291163, x_val=2.121910, 3.370784
iter=5, cost=-0.150000, pred_cost=-0.291904, x_val=2.127269, 3.363985
iter=6, cost=-0.036000, pred_cost=-0.292615, x_val=2.132580, 3.357378
iter=7, cost=-0.224000, pred_cost=-0.293297, x_val=2.137843, 3.350958
iter=8, cost=-0.224000, pred_cost=-0.293951, x_val=2.143058, 3.344720
iter=9, cost=-0.106000, pred_cost=-0.294578, x_val=2.148225, 3.338660
